# Loading CSVs Into MySQL
We got all of our desired data into tabular format, hooray! But now it's stored in a bunch of CSV files while our database sits completely empty. Let's fix that!

This notebook uses a lot of sequentially-dependent operations, so I would recommend either stepping through this notebook sequentially from the top down or clicking the "Run All" button to avoid errors.

In [1]:
'''
Package Imports, Constants, Global Variable

Run this cell to import all the packages we need and define some constants. 
You'll likely need to install any missing packages to your Python environment
with pip or your package manager of choice.
'''

import mysql.connector
from dotenv import load_dotenv
import os

DATA_DIR_PATH = os.path.join('..', 'data')  # Path of data directory relative to this Jupyter Notebook
OUTPUT_DIR_PATH = os.path.join(DATA_DIR_PATH, 'outputs')

To setup Python's connection to a MySQL Server instance, we have to feed it some login credentials. I'm just doing this through a `.env` file instead of leaking my IP address, username, and password directly in my code. We can also specify the database we're going to be working in, which lets us omit a `USE` statement when we run SQL commands. I included it because we're only working in one database for this project.

If the cell runs successfully, we've successfully created a MySQLConnection object that we can use to talk to the MySQL instance

In [2]:
load_dotenv(dotenv_path = '../.env')

db_server = mysql.connector.connect(
  host = os.getenv('HOST'),
  user = os.getenv('USER'),
  password = os.getenv('PASSWORD'),
  database = os.getenv('DB'),
  allow_local_infile = True
)

print(db_server)
db_cursor =  db_server.cursor()

Our database doesn't have any tables to load data into, so let's add a few! Since all of our data is in the CSV files, we can `DROP TABLE IF EXISTS` to get rid of previous loads.

In [3]:
db_cursor.execute('DROP TABLE IF EXISTS points')
db_cursor.execute('DROP TABLE IF EXISTS activities')
db_cursor.execute('DROP TABLE IF EXISTS challenges')
db_cursor.execute('DROP TABLE IF EXISTS users')


db_cursor.execute(
  '''
  CREATE TABLE IF NOT EXISTS users (
    user_id INT PRIMARY KEY,
    email_address VARCHAR(50) NOT NULL,
    first_name VARCHAR(50) NOT NULL,
    last_name VARCHAR(50) NOT NULL,
    description VARCHAR(100) NOT NULL,
    weight FLOAT NOT NULL,
    city VARCHAR(20),
    state VARCHAR(20),
    country VARCHAR(20)
  )
  '''
)

db_cursor.execute(
  # I love multiline strings
  '''
  CREATE TABLE IF NOT EXISTS activities (
    user_id INT NOT NULL,
    activity_id INT PRIMARY KEY,
    name VARCHAR(50) NOT NULL,
    type VARCHAR(20) NOT NULL,
    description VARCHAR(200),
    filename VARCHAR(20) NOT NULL,
    start DATETIME NOT NULL,
    end DATETIME NOT NULL,
    distance_2d FLOAT NOT NULL,
    distance_3d FLOAT,
    avg_speed FLOAT NOT NULL,
    max_speed FLOAT NOT NULL,
    uphill FLOAT NOT NULL,
    downhill FLOAT NOT NULL,
    avg_hr FLOAT,
    min_hr INT,
    max_hr INT,
    avg_cad FLOAT,
    min_cad INT,
    max_cad INT,
    total_kcal INT,
    CONSTRAINT activities_fk_users FOREIGN KEY (user_id) REFERENCES users (user_id)
  )
  '''
)

db_cursor.execute(
  '''
  CREATE TABLE IF NOT EXISTS points (
    seq_num INT NOT NULL,
    activity_id INT NOT NULL,
    latitude DOUBLE NOT NULL,
    longitude DOUBLE NOT NULL,
    elevation FLOAT NOT NULL,
    time DATETIME NOT NULL,
    speed FLOAT NOT NULL,
    hr INT,
    cad INT,
    PRIMARY KEY (seq_num, activity_id),
    CONSTRAINT points_fk_activities FOREIGN KEY (activity_id) REFERENCES activities (activity_id)
  )
  '''
)

db_cursor.execute(
  '''
  CREATE TABLE IF NOT EXISTS challenges (
    challenge_id INT PRIMARY KEY,
    join_datetime DATETIME NOT NULL,
    name VARCHAR(50) NOT NULL,
    completed TINYINT NOT NULL,
    type VARCHAR(20) NOT NULL,
    user_id INT NOT NULL,
    CONSTRAINT challenges_fk_users FOREIGN KEY (user_id) REFERENCES users (user_id)
  )
  '''
)

Time to load CSV files!

In [4]:
def load_csv_to_db(cursor :mysql.connector.cursor, table :str, csv_path : os.PathLike):
  cursor.execute (
    f'''
    LOAD DATA LOCAL INFILE '{csv_path}'
    INTO TABLE {table}
    FIELDS TERMINATED BY ','
    ENCLOSED BY '"'
    LINES TERMINATED BY '\n'
    IGNORE 1 LINES
    '''
)
  
# files = os.listdir(OUTPUT_DIR_PATH)
# for file in files:
#   table_name = file.split('.')[0]
#   csvpath = '../data/outputs/' + file
#   print(f'Loading {file} \n\tInto table {table_name} \n\tFrom file {csvpath}')

#   load_csv_to_db(db_cursor, table_name, csvpath)

files = os.listdir(OUTPUT_DIR_PATH)
my_dict = {}
for file in files:
  table_name = file.split('.')[0]
  csvpath = '../data/outputs/' + file
  my_dict[table_name] = csvpath

print(my_dict)
load_csv_to_db(db_cursor, 'users', my_dict['users'])
load_csv_to_db(db_cursor, 'activities', my_dict['activities'])
load_csv_to_db(db_cursor, 'challenges', my_dict['challenges'])
load_csv_to_db(db_cursor, 'points', my_dict['points'])


{'activities': '../data/outputs/activities.csv', 'challenges': '../data/outputs/challenges.csv', 'points': '../data/outputs/points.csv', 'users': '../data/outputs/users.csv'}


This last cell is very important (I was wondering why nothing was showing up in the database for so long because I forgot the `commit()` call lmao). In the `mysql.connector`, the `connection.commit()` method is used to permanently save any changes made to the database during the current transaction. 

By default, the mysql.connector does not use autocommit, meaning that changes (such as `INSERT`, `UPDATE`, or `DELETE` operations) are staged and held in a temporary state until explicitly committed

In [5]:
db_server.commit()
db_server.close()